In [1]:
import cfscrape
from lxml import etree
import pandas as pd
from bs4 import BeautifulSoup
import csv

'''
get_document_links is the crawler
It is used to retrieve all the links to documents that are returned when performing a search query
The website to crawl should be given as a paramet to the function 
'''

def get_document_links(link): 
    all_links = [[], []]
    page = 0
    offset = 0
        
    website = link
    
    while True: 
        print('scraping page: ' , page) 
        
        scraper = cfscrape.create_scraper()
        scraped_html=scraper.get(website).content.decode("utf-8") 
        parsed_html = BeautifulSoup(scraped_html)
    
        data = parsed_html.body.find_all('div', attrs={'class':'article'})
        if data: 
            for div in data:
                block = div.find('h4')
                links = block.findAll('a')
                for a in links:
                    all_links[0].append(a.text)
                    all_links[1].append("https://www.interpol.int" + a['href'])
            
            if len(links)<1:
                break
                    
            offset = offset + 10
            page = page + 1
            website = 'https://www.interpol.int/contentinterpol/search/(offset)/'+ str(offset) + '?SearchText=cyber+security'
                    
        else:
            print('No page left')
            break

    return all_links

    
        
        
     
        

In [2]:
links = get_document_links('https://www.interpol.int/contentinterpol/search?SearchText=cyber+security')

scraping page:  0
scraping page:  1
scraping page:  2
scraping page:  3
scraping page:  4
scraping page:  5
scraping page:  6
scraping page:  7
scraping page:  8
scraping page:  9
scraping page:  10
scraping page:  11
scraping page:  12
scraping page:  13
scraping page:  14
scraping page:  15
scraping page:  16
scraping page:  17
scraping page:  18
scraping page:  19
scraping page:  20
scraping page:  21
scraping page:  22
scraping page:  23
scraping page:  24
scraping page:  25
scraping page:  26
scraping page:  27
scraping page:  28
scraping page:  29
scraping page:  30
scraping page:  31
scraping page:  32
scraping page:  33
scraping page:  34
scraping page:  35
scraping page:  36
scraping page:  37
scraping page:  38
scraping page:  39
scraping page:  40
scraping page:  41
scraping page:  42
scraping page:  43
scraping page:  44
scraping page:  45
scraping page:  46
scraping page:  47
scraping page:  48
scraping page:  49
scraping page:  50
scraping page:  51
scraping page:  52
scr

In [3]:
'''
This block can be used to write the input to a CSV file. 
'''

import csv

print(len(links[0]))
print(len(links[1]))

csvfile = "/Users/niekkhasuntsev/desktop/scraper.csv"

with open(csvfile, "w") as output:
    writer = csv.writer(output)
    for val in range(len(links[0])):
        writer.writerow([links[0][val], links[1][val]])

1105
1105
[['Major event security experts meet on cyber threats  : DOHA, Qatar - With the cyber environment posing security challenges to major sporting and other public events, global cyber experts have met with...', 'INTERPOL and NEC sign partnership agreement to enhance cyber security  : TOKYO, Japan – In a bid to strengthen the global fight against cybercrime, INTERPOL and Japan’s NEC Corporation today signed a...', '‘Internet of Things’ cyber risks tackled during INTERPOL Digital Security Challenge : VIENNA, Austria – Do you know how to protect your webcam from being hacked? Would you know if a cybercriminal was...', 'Strengthening security in Southern Africa : GABORONE, Botswana – Botswana’s role on the frontline of law enforcement cooperation in Southern Africa was high on the agenda during the first official vis...', "Global policing at a strategic crossroads, says INTERPOL : LYON, France – Senior police officials from around the world are meeting at INTERPOL's eighth Heads of 

In [ ]:
import json

'''
This function can be used to transform the arrays that get_document_links returns
'''

def transform_json(links):
    
    data = {}
   
    for index in range(len(links[0])):
        data[index] = {'title':links[0][index], 'link':links[1][index]}
        print(data)
        
    json_data = json.dumps(data)

    print(json_data)
    return json_data 
    


In [ ]:
transform_json(links)

In [174]:
import cfscrape
from lxml import etree
import pandas as pd
from bs4 import BeautifulSoup
from collections import Counter

'''
scrape_websites is the scraper that is used to retrieve all the data from the links provided by get_document_links
'''

def scrape_websites(links, int1, int2):
    
    irr_data = {}
    rel_data = {}
    
    irr_counter = 0
    rel_counter = 0
    
    categories = []

    #Loop through the list of links and retrieve the data 
    for index in range(int1, int2): 
        scraper = cfscrape.create_scraper()
        try:
            scraped_html=scraper.get(links[1][index]).content.decode("utf-8") 
            parsed_html = BeautifulSoup(scraped_html)
            
            link = links[1][index]
            category = link.split('/')[-3]
            
            categories.append(category)
            
            #Remove the script tags from the parsed html
            [x.extract() for x in parsed_html.findAll('script')]
            titleblock = parsed_html.body.find('h1', attrs={'class':'titre'})
            if titleblock:
                title = parsed_html.body.find('h1', attrs={'class':'titre'}).text
                textblock = parsed_html.body.find('div', attrs={'class':'contextual'})
                if textblock:
                    text = parsed_html.body.find('div', attrs={'class':'contextual'}).text
        
                    website_title = parsed_html.body.find_all('title')
            
                    split = text.split()
                    cyber = split.count('cyber')
                    security = split.count('security')
            
        
                    if cyber > 2:
                        rel_data[rel_counter] = {'title':title, 'text':text, 'cyber':cyber, 'security': security}
                        rel_counter = rel_counter + 1
                        
                    if cyber > 1 and security > 2:
                        rel_data[rel_counter] = {'title':title, 'text':text, 'cyber':cyber, 'security': security}
                        rel_counter = rel_counter + 1

            
                    else:
                        irr_data[irr_counter] = {'title':title, 'text':text, 'cyber':cyber, 'security': security}
                        irr_counter = irr_counter + 1
                
            index = index + 1
            print('loading')
            

        
        except UnicodeDecodeError:
            continue
            
    print(Counter(categories))
        
    all_data = [[irr_data], [rel_data]]
    
    return all_data


        
            
    

In [194]:
data_array = scrape_websites(links, 0, 200)
data_test = scrape_websites(links, 201, 400)
print(len(data_array))
print(len(data_test))
print('done')

loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading


In [195]:
from sklearn import tree
import os # For file reading
import json # For json reading
import re
import numpy as np

from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

In [197]:
names = ["Nearest Neighbors","Naive Bayes", "Neural Network", "Decision Tree", "Random Forest", 
         "Linear SVM", "RBF SVM"]


classifiers = [
    KNeighborsClassifier(3),
    GaussianNB(),
    MLPClassifier(alpha=1),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    SVC(kernel="linear", C=0.025),
    SVC(gamma=2, C=1)
]


RELEVANT_STRING = 'relevant'
IRRELEVANT_STRING = 'not relevant'

X = []
Y = []

print(len(data_array[1][0]))
print(len(data_array[0][0]))

print(len(data_test[1][0]))
print(len(data_test[0][0]))

relevant = data_array[1] 
irrelevant = data_array[0]

test_relevant = data_test[1]
test_irrelevant = data_test[0]

for index in range(len(relevant[0])):
    
    X.append([relevant[0][index].get('cyber'), relevant[0][index].get('security')])
    Y.append(RELEVANT_STRING)
    
      
for index in range(len(irrelevant[0])):
    
    X.append([irrelevant[0][index].get('cyber'), irrelevant[0][index].get('security')])
    Y.append(IRRELEVANT_STRING)

for name, clf in zip(names, classifiers):
    clf = clf.fit(X, Y)
    
    false_negative = 0
    false_positive = 0

    for index in range(len(test_relevant[0])):

        prediction = clf.predict(
                [[test_relevant[0][index].get('cyber'), test_relevant[0][index].get('security')]])

        if not prediction[0] == RELEVANT_STRING:
            false_negative += 1

    for index in range(len(test_irrelevant[0])):

        prediction = clf.predict(
                [[test_irrelevant[0][index].get('cyber'), test_irrelevant[0][index].get('security')]])

        if prediction[0] == RELEVANT_STRING:
            false_positive += 1
    print((true_positive+true_negative)/ len(test_relevant[0]) + len(test_irrelevant[0]))

    print("{}: Accuracy: {}. false negatives: {}. false positives: {}.".format(
        name, 
        str(
            (len(test_relevant[0]) + len(test_irrelevant[0]) - false_negative - false_positive) / (len(test_relevant[0])+ len(test_irrelevant[0]))
        
         ),
        str(false_negative),
        str(false_positive),
    ))

16
116
11
148


NameError: name 'true_positive' is not defined